<a href="https://colab.research.google.com/github/nissi00/machine-learning/blob/main/nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# inspecter la dataset

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! unzip /content/drive/MyDrive/archive.zip -d email #déziper et renommer le dossier sous le nom "email"

Archive:  /content/drive/MyDrive/archive.zip
  inflating: email/SMS_test.csv      
  inflating: email/SMS_train.csv     


In [ ]:
df = pd.concat([pd.read_csv('/content/email/SMS_test.csv',encoding="ISO-8859-1"),
                pd.read_csv('/content/email/SMS_train.csv',encoding="ISO-8859-1")])
#concatener les fichiers pour une répartition es données test et train moins biaisée

In [ ]:
df

,S. No.,Message_body,Label
0,1,"UpgrdCentre Orange customer, you may now claim...",Spam
1,2,"Loan for any purpose £500 - £75,000. Homeowner...",Spam
2,3,Congrats! Nokia 3650 video camera phone is you...,Spam
3,4,URGENT! Your Mobile number has been awarded wi...,Spam
4,5,Someone has contacted our dating service and e...,Spam
...,...,...,...
952,953,hows my favourite person today? r u workin har...,Non-Spam
953,954,How much you got for cleaning,Non-Spam
954,955,Sorry da. I gone mad so many pending works wha...,Non-Spam
955,956,Wat time ü finish?,Non-Spam


In [ ]:
df.Message_body.iloc[3] #  exemple de message

'URGENT! Your Mobile number has been awarded with a £2000 prize GUARANTEED. Call 09058094455 from land line. Claim 3030. Valid 12hrs only'

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1082 entries, 0 to 956
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   S. No.        1082 non-null   int64 
 1   Message_body  1082 non-null   object
 2   Label         1082 non-null   object
dtypes: int64(1), object(2)
memory usage: 33.8+ KB


In [ ]:
df.Label.value_counts(normalize = True)

Non-Spam    0.817006
Spam        0.182994
Name: Label, dtype: float64

# Pré-traitement des données

In [ ]:
df['Label'] = df['Label'].replace(['Non-Spam','Spam'], [1,0])
# renommer "Non-spam" et "Spam" par "1" et "0"
#pour permettre le traitement des données de la variable cible.
# l'entrainement se fait qu'avec des données numériques


In [ ]:
def Clean(Text):
    sms = re.sub('[^a-zA-Z]', ' ', Text) # remplacer les caractère non-alphabétique par l'espace
    sms = sms.lower() #convertir  en minuscule
    sms = sms.split()
    sms = ' '.join(sms)
    return sms


In [ ]:
df["Clean_Text"] = df["Message_body"].apply(Clean)

In [ ]:
print("\033[1m\u001b[45;1m The First 5 Texts after cleaning:\033[0m",*["Clean_Text"][:5], sep = "\n")

 The First 5 Texts after cleaning:
Clean_Text


## tokenisation

c'est segmenter le texte en mots appelé tokens

In [ ]:
df["Tokenize_Text"]=df.apply(lambda row: nltk.word_tokenize(row["Clean_Text"]), axis=1)

print("\033[1m\u001b[45;1m The First 5 Texts after Tokenizing:\033[0m",*df["Tokenize_Text"][:5], sep = "\n")

 The First 5 Texts after Tokenizing:
['upgrdcentre', 'orange', 'customer', 'you', 'may', 'now', 'claim', 'your', 'free', 'camera', 'phone', 'upgrade', 'for', 'your', 'loyalty', 'call', 'now', 'on', 'offer', 'ends', 'th', 'july', 't', 'c', 's', 'apply', 'opt', 'out', 'available']
['loan', 'for', 'any', 'purpose', 'homeowners', 'tenants', 'welcome', 'have', 'you', 'been', 'previously', 'refused', 'we', 'can', 'still', 'help', 'call', 'free', 'or', 'text', 'back', 'help']
['congrats', 'nokia', 'video', 'camera', 'phone', 'is', 'your', 'call', 'calls', 'cost', 'ppm', 'ave', 'call', 'mins', 'vary', 'from', 'mobiles', 'close', 'post', 'bcm', 'ldn', 'wc', 'n', 'xx']
['urgent', 'your', 'mobile', 'number', 'has', 'been', 'awarded', 'with', 'a', 'prize', 'guaranteed', 'call', 'from', 'land', 'line', 'claim', 'valid', 'hrs', 'only']
['someone', 'has', 'contacted', 'our', 'dating', 'service', 'and', 'entered', 'your', 'phone', 'because', 'they', 'fancy', 'you', 'to', 'find', 'out', 'who', 'it', 'i

## retirer les stopwords

les stopwords ou mots vides sont des mots ayant peu de sens dans la compréhension du texte.
Ils sont généralement retirer lors du pré traitement des données pour ne fournir que les données les plus utiles à la machine.

In [ ]:
def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    filtered_text = [word for word in text if word not in stop_words]
    return filtered_text

df["Nostopword_Text"] = df["Tokenize_Text"].apply(remove_stopwords)

print("\033[1m\u001b[45;1m The First 5 Texts after removing the stopwords:\033[0m",*df["Nostopword_Text"][:5], sep = "\n")

 The First 5 Texts after removing the stopwords:
['upgrdcentre', 'orange', 'customer', 'may', 'claim', 'free', 'camera', 'phone', 'upgrade', 'loyalty', 'call', 'offer', 'ends', 'th', 'july', 'c', 'apply', 'opt', 'available']
['loan', 'purpose', 'homeowners', 'tenants', 'welcome', 'previously', 'refused', 'still', 'help', 'call', 'free', 'text', 'back', 'help']
['congrats', 'nokia', 'video', 'camera', 'phone', 'call', 'calls', 'cost', 'ppm', 'ave', 'call', 'mins', 'vary', 'mobiles', 'close', 'post', 'bcm', 'ldn', 'wc', 'n', 'xx']
['urgent', 'mobile', 'number', 'awarded', 'prize', 'guaranteed', 'call', 'land', 'line', 'claim', 'valid', 'hrs']
['someone', 'contacted', 'dating', 'service', 'entered', 'phone', 'fancy', 'find', 'call', 'landline', 'pobox', 'n', 'tf', 'p']


## lemmentisation

Elle vise à ramener les mots à leur forme canonique (lemme) en prenant en compte leur partie du discours et leur contexte grammatical dans la phrase. Le lemme est la forme de base ou le dictionnaire du mot. Par exemple, le lemme du verbe "marche" est "marcher", et le lemme du nom "marches" est "marche".

In [ ]:
lemmatizer = WordNetLemmatizer()
# lemmatize string
def lemmatize_word(text):
    #word_tokens = word_tokenize(text)
    # provide context i.e. part-of-speech
    lemmas = [lemmatizer.lemmatize(word, pos ='v') for word in text]
    return lemmas


df["Lemmatized_Text"] = df["Nostopword_Text"].apply(lemmatize_word)
print("\033[1m\u001b[45;1m The First 5 Texts after lemitization:\033[0m",*df["Lemmatized_Text"][:5], sep = "\n")

 The First 5 Texts after lemitization:
['upgrdcentre', 'orange', 'customer', 'may', 'claim', 'free', 'camera', 'phone', 'upgrade', 'loyalty', 'call', 'offer', 'end', 'th', 'july', 'c', 'apply', 'opt', 'available']
['loan', 'purpose', 'homeowners', 'tenant', 'welcome', 'previously', 'refuse', 'still', 'help', 'call', 'free', 'text', 'back', 'help']
['congrats', 'nokia', 'video', 'camera', 'phone', 'call', 'call', 'cost', 'ppm', 'ave', 'call', 'mins', 'vary', 'mobiles', 'close', 'post', 'bcm', 'ldn', 'wc', 'n', 'xx']
['urgent', 'mobile', 'number', 'award', 'prize', 'guarantee', 'call', 'land', 'line', 'claim', 'valid', 'hrs']
['someone', 'contact', 'date', 'service', 'enter', 'phone', 'fancy', 'find', 'call', 'landline', 'pobox', 'n', 'tf', 'p']


In [ ]:
from sklearn.model_selection import train_test_split # diviser les données en données d'entrainement et de test

In [ ]:
y=df['Label'].values # variable cible
x= df['Lemmatized_Text'].values# variable dépendante

In [ ]:
seed = 123
x_train, x_test, y_train, y_test = train_test_split(
    x,
    y,test_size = 0.2,
    random_state = seed
)

In [ ]:
y

array([0, 0, 0, ..., 1, 1, 1])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# la vectorisation est une technique qui permet de numériser les données
# dans notre cas ce sont les données de la variable dépendante puisque ceux de la variable cible
# ont fait plus haut objet de pré-traitement



In [ ]:
# Avant d'appliquer fit_transform(), convertir la liste de listes en une liste de chaînes de caractères
x_train_str = [' '.join(tokens) for tokens in x_train]
x_test_str = [' '.join(tokens) for tokens in x_test]

# Ensuite, appliquer fit_transform()
Vectorizer = CountVectorizer()
x_train_vec = Vectorizer.fit_transform(x_train_str)
x_test_vec = Vectorizer.transform(x_test_str)


In [ ]:
x_train_vec


<865x2392 sparse matrix of type '<class 'numpy.int64'>'
	with 7230 stored elements in Compressed Sparse Row format>

# entrainement et prédictions

In [ ]:
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
svm_model=SVC()
nb_model=MultinomialNB()
knn_model=KNeighborsClassifier()
rf_model=RandomForestClassifier()
df_model=DecisionTreeClassifier()
lst_model=[svm_model,nb_model,knn_model,rf_model,df_model]
# importation dess modèles pour une étude comparatif

In [ ]:
# entrainement avec différents algorithmes suivi d'un rapport de classification
from sklearn.metrics import confusion_matrix,classification_report
for i in lst_model:
  print(i)
  i.fit(x_train_vec,y_train)
  y_pred=i.predict(x_test_vec)
  print("********************************************************************************")
  print(classification_report(y_test,y_pred))
  print("********************************************************************************")

SVC()
********************************************************************************
              precision    recall  f1-score   support

           0       0.97      0.85      0.91        40
           1       0.97      0.99      0.98       177

    accuracy                           0.97       217
   macro avg       0.97      0.92      0.94       217
weighted avg       0.97      0.97      0.97       217

********************************************************************************
MultinomialNB()
********************************************************************************
              precision    recall  f1-score   support

           0       0.91      1.00      0.95        40
           1       1.00      0.98      0.99       177

    accuracy                           0.98       217
   macro avg       0.95      0.99      0.97       217
weighted avg       0.98      0.98      0.98       217

********************************************************************************


In [ ]:
for i in lst_model:
  print(i)
  re=i.predict(Vectorizer.transform(["URGENT! Congratulations! Your Mobile number has been selected as the winner of a guaranteed £2000 prize. To claim your prize, please contact us at 09058094455 using a landline. Remember to provide the code 3030 when you call. This exclusive offer is valid for a limited time of 12 hours only."]))
  print(re)

SVC()
[0]
MultinomialNB()
[0]
KNeighborsClassifier()
[0]
RandomForestClassifier()
[0]
DecisionTreeClassifier()
[0]


In [ ]:
rf_model.predict(Vectorizer.transform(["click on this link to get an iphone for free"]))

array([1])

# Conclusion

Au terme de ce didacticiel , au vue du rapport de classification les résultas sont puôt satisfaisant. Cependant, la dernière ligne du code qui prédit une phrase ressemblant à un spam est prédite comme non-spam. Alors on peut effectuer une mutitude de combinaison de traitements possible comme:
*  Augmenter les données
*  Analyse du contexte
*  Filtrage bayésien
*   Filtrage bayésien
*   Remplacer les URL des mails spamé par un mot clé. les urls sont souvent indicateur de spam



